PREPARATION DES DONNEES

In [1]:
from google.colab import files
import pandas as pd

# Téléchargement manuel des fichiers CSV
uploaded = files.upload()

# Charger les fichiers dans des DataFrames
train_essays = pd.read_csv("train_essays.csv")
test_essays = pd.read_csv("test_essays.csv")
train_prompts = pd.read_csv("train_prompts.csv")
sample_submission = pd.read_csv("sample_submission.csv")

# Afficher un aperçu des données
print("Train Essays:")
print(train_essays.head())

print("\nTest Essays:")
print(test_essays.head())

print("\nTrain Prompts:")
print(train_prompts.head())

print("\nSample Submission:")
print(sample_submission.head())


Saving sample_submission.csv to sample_submission.csv
Saving test_essays.csv to test_essays.csv
Saving train_essays.csv to train_essays.csv
Saving train_prompts.csv to train_prompts.csv
Train Essays:
         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated  
0          0  
1          0  
2          0  
3          0  
4          0  

Test Essays:
         id  prompt_id          text
0  0000aaaa          2  Aaa bbb ccc.
1  1111bbbb          3  Bbb ccc ddd.
2  2222cccc          4  CCC ddd eee.

Train Prompts:
   prompt_id                       prompt_name  \
0          0       

 Étape 2 : Installer et charger PyTorch & Transformers

In [2]:
# Installer PyTorch et Transformers si nécessaire
!pip install transformers torch torchvision torchaudio --quiet

# Importer les bibliothèques nécessaires
import torch
from transformers import BertTokenizer

# Détecter le GPU (si disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de : {device}")

# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.1 MB/s eta 0:00:00
Utilisation de : cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Étape 3 : Prétraitement et tokenisation des textes

In [4]:
# Fonction pour tokeniser les textes
def tokenize_texts(texts, tokenizer, max_length=512):
    return tokenizer(
        texts.tolist(),  # Transformer la série pandas en liste
        padding=True,  # Ajoute du padding pour uniformiser la taille des séquences
        truncation=True,  # Coupe les séquences trop longues (max_length)
        max_length=max_length,  # Fixe une longueur maximale des séquences
        return_tensors="pt"  # Retourne les tenseurs PyTorch (important pour l'entraînement)
    )

# Appliquer la tokenisation aux textes du train et test set
train_tokens = tokenize_texts(train_essays["text"], tokenizer)
test_tokens = tokenize_texts(test_essays["text"], tokenizer)

# Afficher les clés des tokens générés
print("Train Tokens Keys:", train_tokens.keys())


Train Tokens Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


. Résultat de la tokenisation

In [11]:
print(train_tokens.keys())


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


Étape 4 : Préparer les données pour l'entraînement

1. Créer un Dataset PyTorch

In [13]:
# Nous devons convertir les tokens BERT en un jeu de données PyTorch pour pouvoir les utiliser dans notre GAN.
# PyTorch utilise une classe personnalisée Dataset pour gérer les données d’entrée.

import torch
from torch.utils.data import Dataset, DataLoader

class GANDAIGDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            "input_ids": self.texts["input_ids"][idx],
            "attention_mask": self.texts["attention_mask"][idx],
            "label": torch.tensor(self.labels[idx], dtype=torch.float)
        }


2. Diviser les données en train & test


In [15]:
from sklearn.model_selection import train_test_split

# Extraire uniquement les `input_ids`
input_ids = train_tokens["input_ids"]

# Diviser les données en train (80%) et validation (20%)
train_input_ids, val_input_ids, train_labels, val_labels = train_test_split(
    input_ids, train_essays["generated"].values, test_size=0.2, random_state=42
)

# Créer les datasets PyTorch
train_dataset = GANDAIGDataset({"input_ids": train_input_ids}, train_labels)
val_dataset = GANDAIGDataset({"input_ids": val_input_ids}, val_labels)


In [14]:
# On doit séparer nos données d'entraînement (train_essays) en train et validation.

from sklearn.model_selection import train_test_split

# Diviser les données en train (80%) et validation (20%)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_tokens, train_essays["generated"].values, test_size=0.2, random_state=42
)

# Créer les datasets PyTorch
train_dataset = GANDAIGDataset(train_texts, train_labels)
val_dataset = GANDAIGDataset(val_texts, val_labels)


ValueError: Found input variables with inconsistent numbers of samples: [3, 1378]